In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Municipio','MEDU16'])
df18

,Cod Municipio,MEDU16
0,1100015,Sim
1,1100023,Sim
2,1100031,Sim
3,1100049,Sim
4,1100056,Sim
...,...,...
5565,5222005,Sim
5566,5222054,Sim
5567,5222203,Sim
5568,5222302,Não


In [4]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Educação', usecols=['CodMun','Medu16'])
df21

,CodMun,Medu16
0,1100015,Não
1,1100023,Sim
2,1100031,Sim
3,1100049,Sim
4,1100056,Sim
...,...,...
5565,5222005,Sim
5566,5222054,Sim
5567,5222203,Sim
5568,5222302,Sim


In [5]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


Renomeando as colunas

In [6]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MEDU16':'plano_carreira'}) 
df18

,id_municipio,plano_carreira
0,1100015,Sim
1,1100023,Sim
2,1100031,Sim
3,1100049,Sim
4,1100056,Sim
...,...,...
5565,5222005,Sim
5566,5222054,Sim
5567,5222203,Sim
5568,5222302,Não


In [7]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                           'Medu16':'plano_carreira'}) 
df21

,id_municipio,plano_carreira
0,1100015,Não
1,1100023,Sim
2,1100031,Sim
3,1100049,Sim
4,1100056,Sim
...,...,...
5565,5222005,Sim
5566,5222054,Sim
5567,5222203,Sim
5568,5222302,Sim


In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [11]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [12]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [13]:
df = df.drop(['CodMun'], axis=1) #eliminando coluna repetida

In [14]:
df= df.rename(columns={'UF':'sigla_uf',
                'Cod UF':'cod_uf',
                'Mun':'nome_municipio'}) #padronizando

In [15]:
df['plano_carreira']=np.where(df['plano_carreira']=='Não informou','Sem dados',df['plano_carreira'])

In [16]:
df = df[['ano','sigla_uf','id_municipio','nome_municipio','plano_carreira']] #organizando as colunas


In [22]:
df['plano_carreira'].unique()

array(['Sim', 'Não', 'Sem dados', 'Recusa'], dtype=object)

Subindo para o GBQ

In [18]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ano             11140 non-null  int64 
 1   sigla_uf        11140 non-null  object
 2   id_municipio    11140 non-null  int64 
 3   nome_municipio  11140 non-null  object
 4   plano_carreira  11140 non-null  object
dtypes: int64(2), object(3)
memory usage: 522.2+ KB


In [20]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),       
        bigquery.SchemaField('plano_carreira','STRING',description='Se existe plano de carreira para Magistério')]


In [21]:
table_ref = dataset_ref.table('MUNIC_existencia_plano_carreira_magisterio')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=f57a4130-08df-466f-a351-18f6a0644a06>